# Preprocessing

The definitive notebook describing all of the preprocessing steps we took. 

In [10]:
stock_sample=250

In [11]:
import pandas as pd
import wrds 
import numpy as np

from sklearn.covariance import LedoitWolf

Here, we pull data from WRDS.

In [12]:
'''
#Connection: schneeberger, PW: $4lauSchne!!

conn = wrds.Connection()
db = conn

time_periods = [
    ('2000', '2004', '2000_2004'),
    ('2005', '2009', '2005_2009'),
    ('2010', '2014', '2010_2014'),
    ('2015', '2019', '2015_2019')
]

for start_year, end_year, filename in time_periods:
    cap_query = f"""
      WITH cap_calculation AS (
         SELECT permno, 
               (ABS(prc)/cfacpr) * (shrout * cfacshr) AS market_cap
         FROM crsp.msf
         WHERE date = (SELECT MAX(date) 
                        FROM crsp.msf 
                        WHERE date <= '2005-01-01')
         AND cfacpr != 0
         AND cfacshr != 0
      )
      SELECT permno 
      FROM cap_calculation
      ORDER BY market_cap DESC 
      LIMIT 100
    """
    
    returns_query = f"""
        SELECT a.date, a.ret, b.ticker
        FROM crsp.dsf a
        JOIN crsp.dsenames b 
            ON a.permno = b.permno
            AND a.date BETWEEN b.namedt AND b.nameendt
        WHERE a.permno IN ({cap_query})
            AND a.date BETWEEN '{start_year}-01-01' AND '{end_year}-12-31'
    """
    
    returns = db.raw_sql(returns_query)
    returns.to_csv(f'Final_Data/{filename}_raw.csv', index=False)

db.close()
'''


'\n#Connection: schneeberger, PW: $4lauSchne!!\n\nconn = wrds.Connection()\ndb = conn\n\ntime_periods = [\n    (\'2000\', \'2004\', \'2000_2004\'),\n    (\'2005\', \'2009\', \'2005_2009\'),\n    (\'2010\', \'2014\', \'2010_2014\'),\n    (\'2015\', \'2019\', \'2015_2019\')\n]\n\nfor start_year, end_year, filename in time_periods:\n    cap_query = f"""\n      WITH cap_calculation AS (\n         SELECT permno, \n               (ABS(prc)/cfacpr) * (shrout * cfacshr) AS market_cap\n         FROM crsp.msf\n         WHERE date = (SELECT MAX(date) \n                        FROM crsp.msf \n                        WHERE date <= \'2005-01-01\')\n         AND cfacpr != 0\n         AND cfacshr != 0\n      )\n      SELECT permno \n      FROM cap_calculation\n      ORDER BY market_cap DESC \n      LIMIT 100\n    """\n    \n    returns_query = f"""\n        SELECT a.date, a.ret, b.ticker\n        FROM crsp.dsf a\n        JOIN crsp.dsenames b \n            ON a.permno = b.permno\n            AND a.date

In [13]:
df00 = pd.read_csv('Final_Data/2000_2004_raw.csv')
df05 = pd.read_csv('Final_Data/2005_2009_raw.csv')
df10 = pd.read_csv('Final_Data/2010_2014_raw.csv')
df15 = pd.read_csv('Final_Data/2015_2019_raw.csv')

## Simple Cleaning

Cleaning: remove securities with missing data, convert to log returns

In [14]:
import numpy as np

periods = ['2000_2004', '2005_2009', '2010_2014', '2015_2019']
processed_dfs = {}

for period in periods:
   # Import
   df = pd.read_csv(f'Data/{period}_raw.csv')
   
   # Format columns
   df['date'] = pd.to_datetime(df['date'])
   df['ret'] = pd.to_numeric(df['ret'], errors='coerce')
   df['ticker'] = df['ticker'].astype(str)
   
   # Convert to log returns
   df['log_ret'] = np.log(1 + df['ret'])
   
   # Calculate missing data percentage per ticker
   total_days = df['date'].nunique()
   missing_pct = df.groupby('ticker').size() / total_days
   valid_tickers = missing_pct[missing_pct >= 0.95].index
   
   # Filter and store
   df_clean = df[df['ticker'].isin(valid_tickers)]
   processed_dfs[period] = df_clean
   
   # Save
   df_clean.to_csv(f'Final_Data/{period}_clean.csv', index=False)

In [15]:
df00 = pd.read_csv('Final_Data/2000_2004_clean.csv')
df05 = pd.read_csv('Final_Data/2005_2009_clean.csv')
df10 = pd.read_csv('Final_Data/2010_2014_clean.csv')
df15 = pd.read_csv('Final_Data/2015_2019_clean.csv')

Remove highly correlated stocks

In [16]:
def remove_highly_correlated(df, threshold=0.90):
    # Convert to numeric, coerce errors to NaN
    df_numeric = df.select_dtypes(include=[np.number])
    
    # Calculate correlation matrix
    corr_matrix = df_numeric.corr().abs()
    
    # Create upper triangle mask
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find columns with correlation greater than threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    # Drop the identified columns
    df_cleaned = df.drop(columns=to_drop)
    
    return df_cleaned

# Load and process each dataframe
dfs = {
    '2000_2004': 'Final_Data/2000_2004_clean.csv',
    '2005_2009': 'Final_Data/2005_2009_clean.csv',
    '2010_2014': 'Final_Data/2010_2014_clean.csv',
    '2015_2019': 'Final_Data/2015_2019_clean.csv'
}

for period, file_path in dfs.items():
    # Read CSV
    df = pd.read_csv(file_path)
    
    # Set date as index if it exists as a column
    if 'date' in df.columns:
        df.set_index('date', inplace=True)
    
    # Remove highly correlated securities
    df_cleaned = remove_highly_correlated(df)
    
    # Save cleaned dataframe
    output_path = f'Final_Data/{period}_clean_nomulticoll.csv'
    df_cleaned.to_csv(output_path)

Load and reshape. We had duplicate data issues in 2005, so we clean that out before reshaping.

In [17]:
df00 = pd.read_csv('Final_Data/2000_2004_clean_nomulticoll.csv')
df00 = df00.pivot(index = 'date', columns = 'ticker', values = 'ret')

df05 = pd.read_csv('Final_Data/2005_2009_clean_nomulticoll.csv')

# Drop duplicates based on date and ticker combination
df05 = df05.drop_duplicates(subset=['date', 'ticker'], keep='first')
df05.reset_index(drop=True, inplace=True)
df05 = df05.pivot(index = 'date', columns = 'ticker', values = 'ret')

df10 = pd.read_csv('Final_Data/2010_2014_clean_nomulticoll.csv')
df10 = df10.pivot(index = 'date', columns = 'ticker', values = 'ret')

df15 = pd.read_csv('Final_Data/2015_2019_clean_nomulticoll.csv')
df15 = df15.pivot(index = 'date', columns = 'ticker', values = 'ret')

## Random Subsampling 

In [18]:
from functions import random_subsample2

(df00, df05, df10, df15) = random_subsample2([df00, df05, df10, df15], n_samples=stock_sample, seed=42)

## Imputation

Analyze and impute missing data using a linear interpolation. 

In [19]:
def analyze_missing_data(df):
    """
    Comprehensive missing data analysis for a pandas DataFrame.
    """
    # Basic missing data info
    print("\n=== Missing Data Summary ===")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    
    summary = pd.DataFrame({
        'Missing Values': missing,
        'Missing %': missing_pct,
        'Data Type': df.dtypes
    }).sort_values('Missing %', ascending=False)
    
    print(summary[summary['Missing Values'] > 0])
    
    # Temporal patterns
    if 'date' in df.columns:
        print("\n=== Missing Values Over Time ===")
        missing_by_date = df.groupby('date').isnull().sum()
        print(f"Dates with most missing values:")
        print(missing_by_date.sum(axis=1).sort_values(ascending=False).head())
    
    # Missing value patterns
    print("\n=== Missing Value Patterns ===")
    print(f"Total missing values: {df.isnull().sum().sum()}")
    print(f"Rows with any missing values: {df.isnull().any(axis=1).sum()} ({df.isnull().any(axis=1).sum()/len(df)*100:.2f}%)")
    print(f"Columns with any missing values: {df.isnull().any(axis=0).sum()} ({df.isnull().any(axis=0).sum()/len(df.columns)*100:.2f}%)")
    
    return summary

print(analyze_missing_data(df00))
print(analyze_missing_data(df05))
print(analyze_missing_data(df10))
print(analyze_missing_data(df15))


=== Missing Data Summary ===
        Missing Values  Missing % Data Type
ticker                                     
WLP                 22   1.751592   float64
TGT                 19   1.512739   float64

=== Missing Value Patterns ===
Total missing values: 41
Rows with any missing values: 41 (3.26%)
Columns with any missing values: 2 (0.80%)
        Missing Values  Missing % Data Type
ticker                                     
WLP                 22   1.751592   float64
TGT                 19   1.512739   float64
CA                   0   0.000000   float64
SLE                  0   0.000000   float64
CMVT                 0   0.000000   float64
...                ...        ...       ...
RTN                  0   0.000000   float64
LU                   0   0.000000   float64
EMR                  0   0.000000   float64
PEG                  0   0.000000   float64
BSX                  0   0.000000   float64

[250 rows x 3 columns]

=== Missing Data Summary ===
        Missing Values  Mis

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def impute_returns(df, method='linear', max_gap=100, plot_sample=True):
    """
    Impute missing values in returns data using linear interpolation
    
    Parameters:
    df: pandas DataFrame with datetime index
    method: interpolation method ('linear', 'cubic', 'polynomial')
    max_gap: maximum number of consecutive missing values to interpolate
    plot_sample: whether to plot sample of interpolated values
    """
    
    # Store original missing value locations
    missing_mask = df.isnull()
    
    # Copy original data
    df_imputed = df.copy()
    
    # Interpolate within max_gap limit
    df_imputed = df_imputed.interpolate(
        method=method,
        limit=max_gap,
        limit_direction='both',
        axis=0
    )
    
    # Diagnostic information
    total_missing = missing_mask.sum().sum()
    still_missing = df_imputed.isnull().sum().sum()
    print(f"\nImputation Summary:")
    print(f"Original missing values: {total_missing}")
    print(f"Values imputed: {total_missing - still_missing}")
    print(f"Remaining missing values: {still_missing}")
    
    return df_imputed

df00 = impute_returns(df00)
df05 = impute_returns(df05)
df10 = impute_returns(df10)
df15 = impute_returns(df15)


Imputation Summary:
Original missing values: 41
Values imputed: 41
Remaining missing values: 0

Imputation Summary:
Original missing values: 91
Values imputed: 91
Remaining missing values: 0

Imputation Summary:
Original missing values: 21
Values imputed: 21
Remaining missing values: 0

Imputation Summary:
Original missing values: 150
Values imputed: 150
Remaining missing values: 0


## Splitting into Test Sets
We begin by splitting the data into 20% chunks to get 4 test periods. 

In [21]:
datasets = [df00, df05, df10, df15]
suffixes = ['a', 'b', 'c', 'd', 'e']

for i, df in enumerate(datasets):
    base_name = f'df{str(i*5).zfill(2)}'  # Creates df00, df05, df10, df15
    n = len(df)
    chunk_size = n // 5
    
    for j, suffix in enumerate(suffixes):
        start_idx = j * chunk_size
        end_idx = start_idx + chunk_size if j < 4 else None  # For last chunk, include remainder
        
        # Create new variable name dynamically
        new_df_name = f"{base_name}{suffix}"
        globals()[new_df_name] = df.iloc[start_idx:end_idx].copy()

## VAR Model

For purposes of walk-forward testing, we estimate a VAR (VARMA?!) model on our training set and use the residuals as training data. Then we also use those parameters to take residuals of the test set, and use that data for testing. 

In [23]:
from statsmodels.tsa.api import VAR
import pandas as pd
import os

datasets = {
    "df00": [df00a, df00b, df00c, df00d, df00e],
    "df05": [df05a, df05b, df05c, df05d, df05e],
    "df10": [df10a, df10b, df10c, df10d, df10e],
    "df15": [df15a, df15b, df15c, df15d, df15e]
}

for prefix, chunk_list in datasets.items():
    for i in range(4):
        training = chunk_list[i]
        test = chunk_list[i + 1]

        model = VAR(training)
        results = model.fit(maxlags=1)

        resid_is = results.resid.copy()
        varname_is = f"{prefix}{chr(97 + i)}_varresid_is"
        globals()[varname_is] = resid_is
        filepath_is = f"Final_Data/Walkforward_Sets/{stock_sample}_stocks_seed42/{varname_is}.csv"
        resid_is.to_csv(filepath_is)

        test_lag = test.shift(1)
        test_lag.iloc[0] = training.iloc[-1]

        intercept = results.params.iloc[0]
        lag_coef = results.params.iloc[1]

        predicted = intercept + test_lag.multiply(lag_coef, axis=1)
        resid_os = test - predicted

        varname_os = f"{prefix}{chr(97 + i + 1)}_varresid_os"
        globals()[varname_os] = resid_os
        filepath_os = f"Final_Data/Walkforward_Sets/{stock_sample}_stocks_seed42/{varname_os}.csv"
        resid_os.to_csv(filepath_os)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarnin

## Covariance Matrix Estimation with Ledoit-Wolf

In [24]:
from sklearn.covariance import ledoit_wolf
import numpy as np
import pandas as pd
import os

def cov_to_corr(cov):
    d = np.sqrt(np.diag(cov))
    return cov / np.outer(d, d)

def process_ledoit_wolf_correlations():
    base_prefixes = ['df00', 'df05', 'df10', 'df15']
    letters = ['a', 'b', 'c', 'd', 'e']
    
    for prefix in base_prefixes:
        for i in range(4):
            train_name = f"{prefix}{letters[i]}_varresid_is"
            test_name = f"{prefix}{letters[i+1]}_varresid_os"
            train_df = globals()[train_name]
            test_df = globals()[test_name]
            
            X_train = train_df.values
            X_test = test_df.values
            
            cov_train, shrinkage = ledoit_wolf(X_train)
            corr_train = cov_to_corr(cov_train)
            corr_is_filename = f"Final_Data/Walkforward_Sets/{stock_sample}_stocks_seed42/{train_name}_corr_is.csv"
            pd.DataFrame(corr_train, index=train_df.columns, columns=train_df.columns).to_csv(corr_is_filename)
            
            sample_cov_test = np.cov(X_test, rowvar=False)
            avg_var = np.mean(np.diag(sample_cov_test))
            target = avg_var * np.eye(sample_cov_test.shape[0])
            cov_test = shrinkage * target + (1 - shrinkage) * sample_cov_test
            corr_test = cov_to_corr(cov_test)
            corr_os_filename = f"Final_Data/Walkforward_Sets/{stock_sample}_stocks_seed42/{test_name}_corr_os.csv"
            pd.DataFrame(corr_test, index=test_df.columns, columns=test_df.columns).to_csv(corr_os_filename)

process_ledoit_wolf_correlations()